In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from catboost import CatBoostClassifier, Pool

In [290]:
root_dir = os.path.dirname(os.getcwd())
display(root_dir)
data_dir = os.path.join(root_dir, 'data')
display(data_dir)
train_path = os.path.join(data_dir, 'train.xlsx')
display(train_path)

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\train.xlsx'

In [291]:
df = pd.read_excel(train_path)

In [292]:
display(df.head())

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1


In [293]:
target = df['Дата отмены'].notna().astype(int)
display(target)

0        1
1        0
2        0
3        0
4        0
        ..
26169    0
26170    0
26171    0
26172    1
26173    0
Name: Дата отмены, Length: 26174, dtype: int32

In [294]:
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Номер «Стандарт»,2,3


### Train-test split learning

In [ ]:
def tt_split():
    target = df['Дата отмены'].notna().astype(int)
    display(target)
    
    
    train_df = df.sample(frac=0.8, random_state=42)
    test_df = df.drop(train_df.index)
    display(train_df.shape, test_df.shape)
    
    
    train_target = target[train_df.index]
    test_target = target.drop(train_df.index)
    display(train_target,train_target.shape)
    display(test_target,test_target.shape)
    
    
    train_pool = Pool(train_df, train_target, cat_features=categorical_features)
    test_pool = Pool(test_df, test_target, cat_features=categorical_features)

In [295]:
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)
display(train_df.shape, test_df.shape)

(20939, 12)

(5235, 12)

In [296]:
categorical_features = ["Способ оплаты", "Источник","Категория номера"]

In [297]:
train_target = target[train_df.index]
test_target = target.drop(train_df.index)
display(train_target,train_target.shape)
display(test_target,test_target.shape)


5405     0
20995    0
15439    0
19500    0
17112    0
        ..
17888    1
1806     0
3871     0
3791     1
24555    0
Name: Дата отмены, Length: 20939, dtype: int32

(20939,)

9        0
11       0
13       0
16       1
24       1
        ..
26155    0
26156    0
26163    0
26170    0
26172    1
Name: Дата отмены, Length: 5235, dtype: int32

(5235,)

In [298]:
train_pool = Pool(train_df, train_target, cat_features=categorical_features)
test_pool = Pool(test_df, test_target, cat_features=categorical_features)

### Model init with params {epochs = 100 , depth = 6} - will test

In [358]:
model = CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=6, verbose=True,early_stopping_rounds=500)

In [359]:
model.fit(train_pool,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6115333	total: 101ms	remaining: 8m 26s
1:	learn: 0.5398979	total: 232ms	remaining: 9m 40s
2:	learn: 0.4890657	total: 377ms	remaining: 10m 27s
3:	learn: 0.4477813	total: 516ms	remaining: 10m 44s
4:	learn: 0.4170576	total: 647ms	remaining: 10m 46s
5:	learn: 0.3963912	total: 743ms	remaining: 10m 18s
6:	learn: 0.3800712	total: 853ms	remaining: 10m 8s
7:	learn: 0.3669606	total: 960ms	remaining: 9m 58s
8:	learn: 0.3571257	total: 1.06s	remaining: 9m 48s
9:	learn: 0.3482528	total: 1.16s	remaining: 9m 38s
10:	learn: 0.3412013	total: 1.26s	remaining: 9m 32s
11:	learn: 0.3354012	total: 1.35s	remaining: 9m 23s
12:	learn: 0.3293279	total: 1.45s	remaining: 9m 16s
13:	learn: 0.3255539	total: 1.57s	remaining: 9m 18s
14:	learn: 0.3224932	total: 1.67s	remaining: 9m 14s
15:	learn: 0.3200993	total: 1.78s	remaining: 9m 16s
16:	learn: 0.3174780	total: 1.91s	remaining: 9m 19s
17:	learn: 0.3156590	total: 2.01s	remaining: 9m 17s
18:	learn: 0.3142657	total: 2.12s	remaining: 9m 15s
19:	learn: 0.31305

In [360]:
predictions = model.predict(test_pool)
display(predictions,predictions.shape)
display(test_target)
accuracy = (predictions == test_target).mean()
print(f'Accuracy: {accuracy}')
from sklearn.metrics import roc_auc_score
score = roc_auc_score(predictions,test_target)
display(score)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

(5235,)

9        0
11       0
13       0
16       1
24       1
        ..
26155    0
26156    0
26163    0
26170    0
26172    1
Name: Дата отмены, Length: 5235, dtype: int32

Accuracy: 0.9480420248328558


0.9642144919676896

In [361]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [362]:
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11213,1,50200.0,50200,Банк. карта [Кешбэк. МИР]: Эквайринг TravelLin...,2022-03-27 21:30:38,2022-05-07 15:00:00,2,2022-05-09 12:00:00,Официальный сайт,Номер «Студия»,4,2
11214,1,190100.0,43500,Система быстрых платежей: Эквайринг ComfortBoo...,2023-11-11 17:51:55,2024-02-16 15:00:00,5,2024-02-21 12:00:00,Официальный сайт,Коттедж с 2 спальнями,4,1
11215,1,42300.0,42300,Банк. карта: Банк Россия (банк. карта),2022-02-09 02:23:14,2022-02-09 15:00:00,1,2022-02-10 12:00:00,Официальный сайт,Коттедж с 3 спальнями,5,1
11216,1,27900.0,27900,Банк. карта: Банк Россия (банк. карта),2023-01-21 09:10:17,2023-01-28 15:00:00,1,2023-01-29 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1


In [363]:
res=model.predict(tdf)
display(res)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [364]:
hist, bin_edges = np.histogram(res, bins=2)
print(f"Гистограмма: {hist}")
print(f"Границы интервалов: {bin_edges}")

Гистограмма: [9832 1386]
Границы интервалов: [0.  0.5 1. ]


In [365]:
catboost_res_dir = os.path.join(root_dir, 'catboost_res')

In [366]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"res5000ep6.csv"),index = False, header=None)

# Learn on all data

In [349]:
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=True)

In [350]:
categorical_features = ["Способ оплаты", "Источник","Категория номера"]

In [351]:
df = pd.read_excel(train_path)
target = df['Дата отмены'].notna().astype(int)
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df.head(),df.shape)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1


(26174, 12)

In [352]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [353]:
train_df = df 
test_df = tdf

In [354]:
train_pool = Pool(train_df, target, cat_features=categorical_features)

In [355]:
model.fit(train_pool)

0:	learn: 0.6026205	total: 2.55s	remaining: 21m 14s
1:	learn: 0.5348658	total: 2.8s	remaining: 11m 37s
2:	learn: 0.4817024	total: 4.73s	remaining: 13m 4s
3:	learn: 0.4414625	total: 6.61s	remaining: 13m 39s
4:	learn: 0.4096007	total: 8.13s	remaining: 13m 24s
5:	learn: 0.3879003	total: 9.29s	remaining: 12m 44s
6:	learn: 0.3692069	total: 10.5s	remaining: 12m 17s
7:	learn: 0.3554865	total: 10.5s	remaining: 10m 47s
8:	learn: 0.3425546	total: 11.6s	remaining: 10m 34s
9:	learn: 0.3340029	total: 12.1s	remaining: 9m 52s
10:	learn: 0.3258195	total: 13.2s	remaining: 9m 48s
11:	learn: 0.3207858	total: 13.3s	remaining: 9m
12:	learn: 0.3151960	total: 14.4s	remaining: 9m
13:	learn: 0.3126017	total: 14.5s	remaining: 8m 22s
14:	learn: 0.3088134	total: 15.6s	remaining: 8m 24s
15:	learn: 0.3041867	total: 16.7s	remaining: 8m 25s
16:	learn: 0.3012545	total: 17.9s	remaining: 8m 29s
17:	learn: 0.2994373	total: 19.1s	remaining: 8m 32s
18:	learn: 0.2968914	total: 20.3s	remaining: 8m 34s
19:	learn: 0.2935966	to

In [356]:
res=model.predict(tdf)
display(res)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [357]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"res_all500_15.csv"),index = False, header=None)

## Another preproc
### Hotel to [0,1]

In [433]:
model = CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=6, verbose=True)

In [434]:
categorical_features = ["Способ оплаты", "Источник","weekday"]

In [435]:
df = pd.read_excel(train_path)
target = df['Дата отмены'].notna().astype(int)
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df.head(),df.shape)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1


(26174, 12)

In [436]:
df['Гостиница']=(df['Гостиница'] >= 3).astype(int)

In [437]:
df['Гостиница']=(df['Гостиница'] >= 3).astype(int)
types = ['Стандарт', 'Люкс', 'Апартаменты', 'Студия', 'Коттедж']
for t in types:
    df[t] = df['Категория номера'].str.contains(t).astype(int)
df['month'] = df['Дата бронирования'].apply(lambda x : x.strftime('%m'))
df['weekday'] = df['Дата бронирования'].apply(lambda x : x.strftime('%a'))
df['delta_day'] = (df['Заезд'] - df['Дата бронирования']).dt.days
df['Year'] = df['Дата бронирования'].dt.year
df['Quartal'] = df['Дата бронирования'].dt.quarter
df=df.drop(['Дата бронирования','Заезд','Выезд','Категория номера'],axis = 1)
display(df.columns)

Index(['Номеров', 'Стоимость', 'Внесена предоплата', 'Способ оплаты', 'Ночей',
       'Источник', 'Гостей', 'Гостиница', 'Стандарт', 'Люкс', 'Апартаменты',
       'Студия', 'Коттедж', 'month', 'weekday', 'delta_day', 'Year',
       'Quartal'],
      dtype='object')

In [438]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [439]:
tdf['Гостиница']=(tdf['Гостиница'] >= 3).astype(int)
types = ['Стандарт', 'Люкс', 'Апартаменты', 'Студия', 'Коттедж']
for t in types:
    tdf[t] = tdf['Категория номера'].str.contains(t).astype(int)
tdf['month'] = tdf['Дата бронирования'].apply(lambda x : x.strftime('%m'))
tdf['weekday'] = tdf['Дата бронирования'].apply(lambda x : x.strftime('%a'))
tdf['delta_day'] = (tdf['Заезд'] - tdf['Дата бронирования']).dt.days
tdf['Year'] = tdf['Дата бронирования'].dt.year
tdf['Quartal'] = tdf['Дата бронирования'].dt.quarter
tdf = tdf.drop(['Дата бронирования','Заезд','Выезд','Категория номера'],axis = 1)
display(tdf.head())

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Источник,Гостей,Гостиница,Стандарт,Люкс,Апартаменты,Студия,Коттедж,month,weekday,delta_day,Year,Quartal
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,3,1,0,0,0,1,0,11,Tue,1,2023,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,2,1,1,0,0,0,0,12,Mon,6,2022,4
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,1,Официальный сайт,2,0,1,0,0,0,0,11,Fri,15,2022,4
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),3,Официальный сайт,4,0,0,0,1,0,0,07,Tue,33,2023,3
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,5,Официальный сайт,1,0,1,0,0,0,0,03,Thu,3,2023,1


In [440]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [441]:
train_df = df
test_df = tdf
train_pool = Pool(train_df, target, cat_features=categorical_features)

In [442]:
rf.fit(train_df,target)
feature = pd.DataFrame({'name':df.columns,'featue':rf.feature_importances_}).sort_values('featue',ascending = False)
feature

ValueError: could not convert string to float: 'Внешняя система оплаты'

In [430]:
model.fit(train_pool)

0:	learn: 0.6137646	total: 52.7ms	remaining: 4m 23s
1:	learn: 0.5319408	total: 112ms	remaining: 4m 40s
2:	learn: 0.4817543	total: 161ms	remaining: 4m 28s
3:	learn: 0.4385705	total: 215ms	remaining: 4m 28s
4:	learn: 0.4055131	total: 269ms	remaining: 4m 29s
5:	learn: 0.3846705	total: 351ms	remaining: 4m 51s
6:	learn: 0.3697459	total: 400ms	remaining: 4m 45s
7:	learn: 0.3559089	total: 442ms	remaining: 4m 36s
8:	learn: 0.3431261	total: 493ms	remaining: 4m 33s
9:	learn: 0.3343050	total: 550ms	remaining: 4m 34s
10:	learn: 0.3272750	total: 599ms	remaining: 4m 31s
11:	learn: 0.3217653	total: 649ms	remaining: 4m 29s
12:	learn: 0.3182720	total: 710ms	remaining: 4m 32s
13:	learn: 0.3145274	total: 761ms	remaining: 4m 30s
14:	learn: 0.3103050	total: 816ms	remaining: 4m 31s
15:	learn: 0.3076493	total: 875ms	remaining: 4m 32s
16:	learn: 0.3044865	total: 931ms	remaining: 4m 32s
17:	learn: 0.3027521	total: 985ms	remaining: 4m 32s
18:	learn: 0.3008017	total: 1.03s	remaining: 4m 31s
19:	learn: 0.2999871	

In [431]:
res=model.predict(tdf)
display(res)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [432]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"TopOneHot.csv"),index = False, header=None)

# Data split